# SNe Ia Simulations

The goal of this notebook is to carry out a set of spectral simulations which include SNe Ia templates in the host-galaxy spectra.

#### Interactive queue:
  salloc -N 1 -p debug -L project
  srun -n 1 -c 64 python bgs-sims.py --sim 4 --nproc 64 --zfind

#### Regular queue:
  [see the bgs-sims-zfind.sh script]
  sbatch bgs-sims-zfind.sh
  squeue -u ioannis
  sqs -u ioannis
  
##### Simulation 1:
  Nominal exposure time.
  Fixed r mag (19.5)
  Fixed moon-zenith (30 deg)
  Fixed moon-phase (0.25)
  Vary moon-angle (0-150)


In [1]:
import os
import numpy as np

import healpy as hp
from astropy.table import Table

In [2]:
import desisim.io
import desispec.io
import desimodel.io 

from desisim.obs import new_exposure
from desisim.scripts import quickgen
from desisim.templates import specify_galparams_dict

In [3]:
#import desimodel.io
#import desimodel.footprint
#from redrock.external.desi import rrdesi
#from desispec.parallel import use_mpi
#import desispec.scripts.group_spectra as group_spectra

In [4]:
%matplotlib inline

### Simulation number and input priors

In [5]:
sim = 1
program = 'BGS'

seed = 678245
nexp = 5
nspec = 3
nreps = 2

In [6]:
seeing_range = [1.0, 1.0]
airmass_range = [1.0, 1.0]
sne_rfluxratiorange = [0.1, 1.0]

In [7]:
moon_phase_range = [0.25, 0.25]
rmagrange = [19.5, 19.5]
zrange = [0.1, 0.3]
moon_zenith_range = [30, 30]
exptime_range = [300,300]
moon_angle_range = [0, 150]

In [8]:
simint = int(sim)
simname = '{:02d}'.format(simint)
night = '2017'+simname+simname
expids = np.arange(nexp*nreps) + simint*1000
print('Simulation name = {}, Night = {}, Number of exposures = {}'.format(simname, night, nexp))
print('Exposure IDs = {}'.format(expids))

Simulation name = 01, Night = 20170101, Number of exposures = 5
Exposure IDs = [1000 1001 1002 1003 1004 1005 1006 1007 1008 1009]


### Define the I/O simulation directories.

In [9]:
os.environ['PIXPROD'] = 'snesim-{}'.format(simname)
os.environ['SPECPROD'] = 'snesim-{}'.format(simname)
os.environ['DESI_SPECTRO_DATA'] = desisim.io.simdir()

print('Simulated raw data will be written to {}'.format(desisim.io.simdir()))
print('Pipeline will read raw data from {}'.format(desispec.io.rawdata_root()))
print('Pipeline will write processed data to {}'.format(desispec.io.specprod_root()))

Simulated raw data will be written to /Users/ioannis/work/desi/spectro/sim/snesim-01/
Pipeline will read raw data from /Users/ioannis/work/desi/spectro/sim/snesim-01/
Pipeline will write processed data to /Users/ioannis/work/desi/spectro/redux/snesim-01


### Assign exposures to different tiles.

This step is necessary to keep the spectra from all stacking on the default tile, and ensures that each tile will have a dedicated set of observing conditions.

In [10]:
tiles = desimodel.io.load_tiles()
xyz = hp.ang2vec(tiles['ra'], tiles['dec'], lonlat=True)
center = hp.ang2vec(130, 18, lonlat=True)
d2 = ((xyz - center)**2).sum(axis=1)
tiles = tiles[np.argsort(d2)[0:nexp]]
print('Selecting the following tiles:')
Table(tiles)

Selecting the following tiles:


TILEID,RA,DEC,PASS,IN_DESI,EBV_MED,AIRMASS,STAR_DENSITY,EXPOSEFAC,PROGRAM,OBSCONDITIONS
int32,float64,float64,int16,int16,float32,float32,float32,float32,str6,int32
22594,130.08,18.03,3,1,0.0255654,1.06033,2664.73,1.25706,DARK,1
40476,129.32,17.9,7,1,0.0281447,1.06097,2753.01,1.27791,BRIGHT,4
16832,128.99,18.05,2,1,0.0298504,1.06023,2795.41,1.29012,DARK,1
28356,131.17,18.02,4,1,0.0231459,1.06038,2514.4,1.23876,GRAY,2
11069,130.26,16.58,1,1,0.0251414,1.06783,2711.23,1.26491,DARK,1


### Generate nexp uniform samplings of the simulation parameters.

In [11]:
rand = np.random.RandomState(seed)

In [12]:
seeds = rand.randint(2**32, size=nexp)
seeings = rand.uniform(seeing_range[0], seeing_range[1], nexp)
exptimes = rand.uniform(exptime_range[0], exptime_range[1], nexp)
airmasses = rand.uniform(airmass_range[0], airmass_range[1], nexp)
moonphases = rand.uniform(moon_phase_range[0], moon_phase_range[1], nexp)
moonangles = rand.uniform(moon_angle_range[0], moon_angle_range[1], nexp)
moonzeniths = rand.uniform(moon_zenith_range[0], moon_zenith_range[1], nexp)

In [13]:
allobj_specs = dict()
allobj_specs[program] = specify_galparams_dict(
    program, 
    zrange=(zrange[0], zrange[1]), 
    magrange=(rmagrange[0],rmagrange[1]))
print(allobj_specs)

{'BGS': {'rmagrange': (19.5, 19.5), 'zrange': (0.1, 0.3)}}


In [14]:
for ii in range(1):
    obsconditions = {'SEEING' : seeings[ii], 'EXPTIME': exptimes[ii], 'AIRMASS': airmasses[ii],
                     'MOONFRAC': moonphases[ii], 'MOONALT': moonzeniths[ii], 'MOONSEP': moonangles[ii]}
    
    sim, fibermap, meta, obsconditions = new_exposure(program=program, seed=seeds[ii],
                                                      nspec=nspec, night=night,
                                                      expid=expids[ii], tileid=tiles[ii][0],
                                                      obsconditions=obsconditions, 
                                                      specify_targets=allobj_specs,
                                                      overwrite=True)

DEBUG:obs.py:101:new_exposure: Generating 3 targets
DEBUG:targets.py:195:get_targets_parallel: Not Parallelizing get_targets for only 3 targets
DEBUG:targets.py:262:get_targets: Using random seed 1800636030
INFO:io.py:787:read_basis_templates: Reading /Users/ioannis/work/desi/spectro/templates/basis_templates/v2.3/star_templates_v2.1.fits
DEBUG:simexp.py:371:simulate_spectra: loading specsim desi config
DEBUG:simexp.py:375:simulate_spectra: creating specsim desi simulator
DEBUG:specsim.py:39:get_simulator: Creating new  Simulator


/usr/local/anaconda3/envs/desi/lib/python3.5/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


INFO:simexp.py:396:simulate_spectra: MJD not in obsconditions, using DATE-OBS 2009-06-18T12:00:00.000
DEBUG:simexp.py:400:simulate_spectra: obsconditions MOONALT = 30.0
DEBUG:simexp.py:400:simulate_spectra: obsconditions EXPTIME = 300.0
DEBUG:simexp.py:400:simulate_spectra: obsconditions MOONFRAC = 0.25
DEBUG:simexp.py:400:simulate_spectra: obsconditions MOONSEP = 122.14476512451438
DEBUG:simexp.py:400:simulate_spectra: obsconditions AIRMASS = 1.0
DEBUG:simexp.py:400:simulate_spectra: obsconditions SEEING = 1.0
DEBUG:simexp.py:518:get_source_types: sky 2 targets
DEBUG:simexp.py:518:get_source_types: star 1 targets
DEBUG:simexp.py:436:simulate_spectra: running simulation
INFO:io.py:147:write_simspec: DATE-OBS 2017-01-01T22:00:00 UTC
INFO:io.py:223:write_simspec: Writing /Users/ioannis/work/desi/spectro/sim/snesim-01/20170101/simspec-00001000.fits


INFO:obs.py:211:new_exposure: Wrote /Users/ioannis/work/desi/spectro/sim/snesim-01/20170101/fibermap-00001000.fits


IntegrityError: datatype mismatch